In [227]:
import torch
from torchvision import datasets, transforms
import numpy as np
from opacus import PrivacyEngine
from tqdm import tqdm

In [299]:
train_loader = torch.utils.data.DataLoader(datasets.MNIST('../mnist',
               train=True, download=True,
               transform=transforms.Compose([transforms.ToTensor(),
               transforms.Normalize((0.1307,), (0.3081,)),]),),
               batch_size=256, shuffle=True, num_workers=1,
               pin_memory=True)

test_loader = torch.utils.data.DataLoader(datasets.MNIST('../mnist',
              train=False,
              transform=transforms.Compose([transforms.ToTensor(),
              transforms.Normalize((0.1307,), (0.3081,)),]),),
              batch_size=1024, shuffle=True, num_workers=1,
              pin_memory=True)

In [300]:
model = torch.nn.Sequential(torch.nn.Conv2d(1, 16, 8, 2, padding=3),
                            torch.nn.ReLU(),
                            torch.nn.MaxPool2d(2, 1), 
                            torch.nn.Conv2d(16, 32, 4, 2), 
                            torch.nn.ReLU(), 
                            torch.nn.MaxPool2d(2, 1), 
                            torch.nn.Flatten(), 
                            torch.nn.Linear(32 * 4 * 4, 32), 
                            torch.nn.ReLU(), 
                            torch.nn.Linear(32, 10))

optimizer = torch.optim.Adadelta(model.parameters())#, lr=0.05)

In [301]:
UPSTREAM_GRAD_BOUND = 0.001
INPUT_BOUND = 2.

In [302]:
def clamp_grad(self, grad_input, grad_output):
#     if grad_input[0] != None:
#         print('BACKWARD max:', grad_input[0].abs().max().item(), 
#               'mean:', grad_input[0].abs().mean().item(), 
#               'shape:', grad_input[0].shape)

    return tuple([None if x == None else x.clamp(-UPSTREAM_GRAD_BOUND, UPSTREAM_GRAD_BOUND) for x in grad_input])

In [303]:
def clamp_input(self, input):
#     print('FORWARD max:', input[0].abs().max().item(), 
#           'mean:', input[0].abs().mean().item(), 
#           'shape:', input[0].shape)
    return tuple([x.clamp(-INPUT_BOUND, INPUT_BOUND) for x in input])

In [304]:
for x in model:
    x.register_backward_hook(clamp_grad)
    x.register_forward_pre_hook(clamp_input)

In [305]:
privacy_engine = PrivacyEngine(model, 
                               batch_size=256, 
                               sample_size=60000,  
                               alphas=range(2,32), 
                               noise_multiplier=1.0, 
                               max_grad_norm=0.1,)

privacy_engine.attach(optimizer)

/anaconda3/envs/torch/lib/python3.8/site-packages/opacus/privacy_engine.py:638: UserWarning: The sample rate will be defined from ``batch_size`` and ``sample_size``.The returned privacy budget will be incorrect.
  warnings.warn(
/anaconda3/envs/torch/lib/python3.8/site-packages/opacus/privacy_engine.py:229: UserWarning: Secure RNG turned off. This is perfectly fine for experimentation as it allows for much faster training performance, but remember to turn it on and retrain one last time before production with ``secure_rng`` turned on.
  warnings.warn(


In [306]:
def train(model, train_loader, optimizer, epoch, device, delta):
    model.train()
    criterion = torch.nn.CrossEntropyLoss()
    losses = []
    for _batch_idx, (data, target) in enumerate(tqdm(train_loader)):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        losses.append(loss.item())
    
    epsilon, best_alpha = optimizer.privacy_engine.get_privacy_spent(delta)
        
    print(
        f"Train Epoch: {epoch} \t"
        f"Loss: {np.mean(losses):.6f} "
        f"(ε = {epsilon:.2f}, δ = {delta}) for α = {best_alpha}")
    
for epoch in range(1, 11):
    train(model, train_loader, optimizer, epoch, device="cpu", delta=1e-5)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 235/235 [00:41<00:00,  5.73it/s]


Train Epoch: 1 	Loss: 1.940043 (ε = 0.96, δ = 1e-05) for α = 10.0


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 235/235 [00:39<00:00,  5.93it/s]


Train Epoch: 2 	Loss: 1.137796 (ε = 1.01, δ = 1e-05) for α = 10.0


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 235/235 [00:40<00:00,  5.74it/s]


...

Train Epoch: 8 	Loss: 0.226614 (ε = 1.27, δ = 1e-05) for α = 10.0


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 235/235 [00:41<00:00,  5.72it/s]


Train Epoch: 9 	Loss: 0.212854 (ε = 1.31, δ = 1e-05) for α = 10.0


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 235/235 [00:42<00:00,  5.48it/s]

Train Epoch: 10 	Loss: 0.202363 (ε = 1.35, δ = 1e-05) for α = 10.0


In [307]:
def test(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data, target
            output = model(data)
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    print('\nTest set: Accuracy: {}/{} ({:.0f}%)\n'.format(
        correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [308]:
test(model, test_loader)


Test set: Accuracy: 9479/10000 (95%)

